In [2]:
"""
 GTSAM Copyright 2010-2018, Georgia Tech Research Corporation,
 Atlanta, Georgia 30332-0415
 All Rights Reserved
 Authors: Frank Dellaert, et al. (see THANKS for the full author list)
 
 See LICENSE for the license information
 
 Simple robotics example using odometry measurements and bearing-range (laser) measurements
 Author: Alex Cunningham (C++), Kevin Deng & Frank Dellaert (Python)
"""

import gtsam
import numpy as np
 
# Create noise models
PRIOR_NOISE = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.3, 0.3, 0.1]))
ODOMETRY_NOISE = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.2, 0.2, 0.1]))
MEASUREMENT_NOISE = gtsam.noiseModel_Diagonal.Sigmas(np.array([0.1, 0.2]))

# Create an empty nonlinear factor graph
graph = gtsam.NonlinearFactorGraph()

# Create the keys corresponding to unknown variables in the factor graph

# Add a prior on pose X1 at the origin. A prior factor consists of a mean and a noise model
graph.add(gtsam.PriorFactorPose2(1, gtsam.Pose2(0.0, 0.0, 0.0), PRIOR_NOISE))

# Add odometry factors between X1,X2 and X2,X3, respectively
graph.add(gtsam.BetweenFactorPose2(
 1,2, gtsam.Pose2(2.0, 0.0, 0.0), ODOMETRY_NOISE))
graph.add(gtsam.BetweenFactorPose2(
 2, 3, gtsam.Pose2(2.0, 0.0, 0.0), ODOMETRY_NOISE))

# Add Range-Bearing measurements to two different landmarks L1 and L2
graph.add(gtsam.BearingRangeFactor2D(
 1, 101, gtsam.Rot2.fromDegrees(45), np.sqrt(4.0+4.0), MEASUREMENT_NOISE))
graph.add(gtsam.BearingRangeFactor2D(
 2, 101, gtsam.Rot2.fromDegrees(90), 2.0, MEASUREMENT_NOISE))
graph.add(gtsam.BearingRangeFactor2D(
 3, 102, gtsam.Rot2.fromDegrees(90), 2.0, MEASUREMENT_NOISE))

# Print graph
print("Factor Graph:\n{}".format(graph))

# Create (deliberately inaccurate) initial estimate
initial_estimate = gtsam.Values()
initial_estimate.insert(1, gtsam.Pose2(-0.25, 0.20, 0.15))
initial_estimate.insert(2, gtsam.Pose2(2.30, 0.10, -0.20))
initial_estimate.insert(3, gtsam.Pose2(4.10, 0.10, 0.10))
initial_estimate.insert(101, gtsam.Point2(1.80, 2.10))
initial_estimate.insert(102, gtsam.Point2(4.10, 1.80))

# Print
print("Initial Estimate:\n{}".format(initial_estimate))

# Optimize using Levenberg-Marquardt optimization. The optimizer
# accepts an optional set of configuration parameters, controlling
# things like convergence criteria, the type of linear system solver
# to use, and the amount of information displayed during optimization.
# Here we will use the default set of parameters.  See the
# documentation for the full set of parameters.
params = gtsam.LevenbergMarquardtParams()
optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
print("\nFinal Result:\n{}".format(result))

# Calculate and print marginal covariances for all variables
marginals = gtsam.Marginals(graph, result)
for (key, str) in [(1, "X1"), (2, "X2"), (3, "X3"), (101, "L1"), (102, "L2")]:
    print("{} covariance:\n{}\n".format(str, marginals.marginalCovariance(key)))

Factor Graph:
size: 6

Factor 0: PriorFactor on 1
  prior mean: (0, 0, 0)
  noise model: diagonal sigmas[0.3; 0.3; 0.1];

Factor 1: BetweenFactor(1,2)
  measured: (2, 0, 0)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 2: BetweenFactor(2,3)
  measured: (2, 0, 0)
  noise model: diagonal sigmas[0.2; 0.2; 0.1];

Factor 3: BearingRangeFactor
Factor 3:   keys = { 1 101 }
  noise model: diagonal sigmas[0.1; 0.2];
ExpressionFactor with measurement: bearing : 0.785398163

Factor 4: BearingRangeFactor
Factor 4:   keys = { 2 101 }
  noise model: diagonal sigmas[0.1; 0.2];
ExpressionFactor with measurement: bearing : 1.57079633

Factor 5: BearingRangeFactor
Factor 5:   keys = { 3 102 }
  noise model: diagonal sigmas[0.1; 0.2];
ExpressionFactor with measurement: bearing : 1.57079633


Initial Estimate:
Values with 5 values:
Value 1: (N5gtsam5Pose2E) (-0.25, 0.2, 0.15)

Value 2: (N5gtsam5Pose2E) (2.3, 0.1, -0.2)

Value 3: (N5gtsam5Pose2E) (4.1, 0.1, 0.1)

Value 101: (N5gtsam6Point2E) (1.8,